# Pipeline : 
### data processing of the manually annoted dataset
### image processing : extraction of RGB volumes
### classifier based on WND-charm Model
### PCA & RDF
### Build a classifier
### Analyse False Positives
### Prediction of Neurons, Astrocytes, Background in ROI's clone

In [ ]:
import numpy as np
import pandas as pd
from functools import partial
import multiprocessing
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as cm
import pickle
import time
from pandas import DataFrame, read_csv,concat,merge

In [ ]:
%run ./configure.ipynb

In [ ]:
%run ../Connection_to_server.ipynb

In [ ]:
imageId = 451
weight=25
height=25

## Pre-pocessing of the manually annotated Dataset

In [ ]:
import Pre_Processing_the_annotated_dataset as pp
reload(pp);

In [ ]:
dfpoints1=read_csv(projectpath+'io/Input/Bloc01_pointes_first_mod.csv')


In [ ]:
# Bloc 2
dfpoints2=read_csv(projectpath+'io/Input/Bloc02_pointes_second.csv')

In [ ]:
import Pre_Classification_process as pcp
reload(pcp);

In [ ]:
filename_Features_bloc1=projectpath+'io/Output/Features_bloc1.npy'

In [ ]:
Features_bloc1=np.load(filename_Features_bloc1)

In [ ]:
Data_FRAMES_bloc1=pd.read_pickle(projectpath+'io/Output/Data_FRAMES_bloc1.pkl')

In [ ]:
filename_Features_bloc2=projectpath+'io/Output/Features_bloc2.npy'

In [ ]:
Features_bloc2=np.load(filename_Features_bloc2)

In [ ]:
Data_FRAMES_bloc2=pd.read_pickle(projectpath+'io/Output/Data_FRAMES_bloc2.pkl')

# Create the full data Frame ( 1135 Adtrocytes, 1135 Neurons, 1151 background) 

In [ ]:
import join_two_blocs as jt
reload(jt);
import Fonctions_used_for_many_DF_pre_processing as fct_pp
reload(fct_pp);

In [ ]:
Data_FRAMES=pd.read_pickle(projectpath+'io/Output/Dataframe_.pkl')

In [ ]:
filename_Features_two_blocs=projectpath+'io/Output/Features_two_blocs.npy'

In [ ]:
Features_two_blocs=np.load(filename_Features_two_blocs)

In [ ]:
def Get_true_y(DF):
    y=np.zeros([DF.shape[0]])
    for j in range(0,DF.shape[0]):
        if (DF.TYPE[j]==1):
            y[j]=1
        elif (DF.TYPE[j]==2):
            y[j]=2
        else :
            y[j]=3
        yr=(y.reshape(-1,1)).ravel() 
            #yr = yr.ravel()   
    return(yr)

In [ ]:
yr_dataFrame=Get_true_y(Data_FRAMES)

In [ ]:
ind_Astro=np.where(yr_dataFrame==1)
ind_Neuro=np.where(yr_dataFrame==2)
ind_Backg=np.where(yr_dataFrame==3)

cor_FV_Astro=Features_two_blocs[ind_Astro]
cor_FV_Neuro=Features_two_blocs[ind_Neuro]
cor_FV_Backg=Features_two_blocs[ind_Backg]

In [ ]:
w=25
h=25
columns_names = list(Data_FRAMES.columns.values)
df_New=np.zeros([2*Data_FRAMES.shape[0],len(columns_names)])
df_New=pd.DataFrame(data=df_New,columns= Data_FRAMES.columns)
df_New=pd.concat([Data_FRAMES, Data_FRAMES], axis=0)
df_New=fct_pp.getDF(df_New)
df_New=fct_pp.Switch_Indices(df_New)

# left up
for i in range(1,len(df_New)/2-1):
    df_New.X[i+Data_FRAMES.shape[0]]=-w/2+df_New.X[i-1]+1
    df_New.Y[i+Data_FRAMES.shape[0]]=-h/2+df_New.Y[i-1]+1

In [ ]:
#  right up
df_New_r_u=Data_FRAMES.copy()
df_New_r_u=pd.DataFrame(data=df_New_r_u,columns= Data_FRAMES.columns)
for i in range(0,len(df_New)/2-1):
    df_New_r_u.X[i]=w/2+df_New.X[i]
    df_New_r_u.Y[i]=-h/2+df_New.Y[i]+1
    


In [ ]:
df_NEW=pd.concat([df_New, df_New_r_u], axis=0)
df_NEW=pd.concat([df_NEW, df_New_l_d], axis=0)
df_NEW=pd.concat([df_NEW, df_New_r_d], axis=0)
df_New=fct_pp.getDF(df_NEW)
df_New=fct_pp.Switch_Indices(df_New)
df_New_copy=df_New.copy()


In [ ]:
df_New_r_u.TYPE[0]

In [ ]:
print df_New_copy.shape
print 5*len(Data_FRAMES)

In [ ]:
print df_New_r_d.Y[0]-h/2
print df_New.Y[0]

In [ ]:
Data_FRAMES.loc[4,:]

In [ ]:
df_New.head()

# PCA & Random Forest Classification

In [ ]:
%%time
%run ./Pipeline_PCA_RANDOM_Forest_Copy1.ipynb
# CPU times: user 16min 53s, sys: 1h 52s, total: 1h 17min 45s
# Wall time: 6min 58s
#CPU times: user 2h 17min 17s, sys: 2h 13min 33s, total: 4h 30min 50s
#Wall time: 25min 13s

# Imshow specific images to understand non differentiation in PCA model

# process in order to display false predicted images

In [ ]:
(fail_img_pred_Neurons,fail_img_pred_Astrocytes,fail_img_pred_Background)=imf.Get_IMG_failed_prediction(yr,predictedVAL)

In [ ]:
sns.reset_orig() # sns.set()

# test prediction sur  1 clone

In [ ]:
import Get_coordinates as gC
reload(gC);

In [ ]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()

image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

size_x=image.getSizeX()
size_y=image.getSizeY()
size_z = image.getSizeZ()
size_c = image.getSizeC()
size_t = image.getSizeT()

large= weight  
nidY=size_x/(int(large/int(large/2)))-3
delta_x=2
z=103

In [ ]:
# Coordinates of the ROI in the original image
x_min=48+367
y_min=2064+646
w=615-367
h=920-646
x_max=x_min+w
y_max=y_min+h

In [ ]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()
imageId=451
image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

pxobj_CH1 = pixels.getTile(theZ=103, theC=0, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH2 = pixels.getTile(theZ=103, theC=1, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH3 = pixels.getTile(theZ=103, theC=2, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))

# Get the normalized original image
pxobj=(pxobj_CH1+pxobj_CH2+pxobj_CH3)/3

In [ ]:
partial_getDATA_coord = partial(gC.get_coordinates_test,imageId=imageId, large=large,delta_x= delta_x,z=z,x_min=x_min,y_min=y_min,x_max=x_max, y_max=y_max)

In [ ]:
filename_resultat =projectpath+'io/Output/resultat.npy'

In [ ]:
res=np.load(filename_resultat)
M_Mv=res[:,0]
liste_coordonnees=res[:,1]


In [ ]:
# Reshape the list of coordinates
b=np.vstack(liste_coordonnees)

x_from_liste_coordonnees=b[:,0]
y_from_liste_coordonnees=b[:,1]
ind_lign_from_liste_coordonnees=b[:,2]
ind_col_from_liste_coordonnees=b[:,3]

In [ ]:
Matrix_TEST=np.vstack(M_Mv)
Matrix_TEST=Matrix_TEST/255.0

In [ ]:
coords=np.where((np.hstack(x_from_liste_coordonnees)>=x_min) & (np.hstack(x_from_liste_coordonnees)<=x_max) & (np.hstack(y_from_liste_coordonnees)>=y_min) & (np.hstack(y_from_liste_coordonnees)<=y_max))[0]

In [ ]:
Im_to_convert_GRAY=Matrix_TEST  #[coords]

In [ ]:
filename_Im_to_convert_GRAY=projectpath+'io/Output/Im_to_convert_GRAY.npy'

In [ ]:
np.save(filename_Im_to_convert_GRAY,Im_to_convert_GRAY)

In [ ]:
Im_to_convert_GRAY=np.load(filename_Im_to_convert_GRAY)

## Process for one tile ipynb

In [ ]:
import process_for_one_tile as pt
reload(pt);

In [ ]:
nb_features=2919

In [ ]:
size_=Im_to_convert_GRAY.shape[0]-1
nidx=Im_to_convert_GRAY.shape[0]

In [ ]:
partial_get_Get_proba = partial(pt.Get_proba, h=height, w=weight, M_mean=Im_to_convert_GRAY, nb_features=nb_features)

In [ ]:
pool = multiprocessing.Pool(27)
try:
    Im_Proba = pool.map(partial_get_Get_proba,xrange(size_))
finally:
    pool.close()

In [ ]:
filename_Im_Proba =projectpath+'io/Output/Im_Proba.npy'

In [ ]:
################################"

In [ ]:
Im_Proba=np.load(filename_Im_Proba)

In [ ]:
Im_proba=np.zeros([Im_Proba.shape[0], Im_Proba.shape[2]])

for g in range(0,Im_Proba.shape[0]):
    Im_proba[g]=Im_Proba[g][g]

In [ ]:
filename_Im_proba =projectpath+'io/Output/Im_proba.npy'

In [ ]:
Im_proba=np.save(filename_Im_proba,Im_proba)

In [ ]:
Im_proba=np.load(filename_Im_proba)

In [ ]:
Probability_img=np.zeros([Im_proba.shape[0]+1,Im_proba.shape[1]])
Probability_img[:Im_proba.shape[0]]=Im_proba
Probability_img[Im_proba.shape[0]]=Probability_img[Im_proba.shape[0]-1]

In [ ]:
conn = BlitzGateway('tbacoyannis','d33pl34rn1ng',port=4064,host='chinensis.polytechnique.fr')
connected = conn.connect()
imageId=451
image = conn.getObject("Image", imageId)
pixels = image.getPrimaryPixels() # get raw pixels information

pxobj_CH1 = pixels.getTile(theZ=103, theC=0, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH2 = pixels.getTile(theZ=103, theC=1, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))
pxobj_CH3 = pixels.getTile(theZ=103, theC=2, theT=0, tile=(x_min,y_min, x_max-x_min , y_max-y_min ))

# Get the normalized original image
pxobj=(pxobj_CH1+pxobj_CH2+pxobj_CH3)/3

In [ ]:
e=124
CH1=Probability_img[:,0]
CH1=np.reshape(CH1, (-1, e) )
CH1=CH1[:,:, np.newaxis]
#     plt.title('Proba Astrocytes')

CH2=Probability_img[:,1]
CH2=np.reshape(CH2,(-1,e))
CH2=CH2[:,:,np.newaxis]
#     plt.title('Proba Neurons')


CH3=Probability_img[:,2]
CH3=np.reshape(CH3, (-1, e) )
CH3=CH3[:,:,np.newaxis]
#     plt.title('Proba Background')

Z=np.dstack([CH1,CH2,CH3])

In [ ]:
filename_RGB_proba_im =projectpath+'io/Output/RGB_proba_im.npy'

In [ ]:
np.save(filename_RGB_proba_im,Z)

In [ ]:
Z=np.load(filename_RGB_proba_im)

In [ ]:
import IM_Display_RGB as imd
reload(imd);

In [ ]:
imd.imshowRGB(Z,pxobj,10)